In [1]:
# From pipeline textbook chapter, adapted for full accident control data
import pandas as pd


In [2]:

from sklearn.preprocessing import LabelEncoder
X_train = pd.read_csv("../Data_Cleaned/train_data/control_2class_X_train.csv")
y_train = pd.read_csv("../Data_Cleaned/train_data/control_2class_y_train.csv")
X_test = pd.read_csv("../Data_Cleaned/test_data/control_2class_X_test.csv")
y_test = pd.read_csv("../Data_Cleaned/test_data/control_2class_y_test.csv")

# Fix np formatting error
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

le = LabelEncoder()
y_train = le.fit_transform(y_train)

le.classes_
print(y_train)
print(X_train)


[0 0 0 ... 0 1 0]
            Street  Temperature(F)  Humidity(%)  Pressure(in)  Visibility(mi)  \
0          Highway            75.0         82.0         29.88            10.0   
1          Highway            45.0         89.0         29.44            10.0   
2       Interstate            91.0         66.0         29.95            10.0   
3            local            80.1         71.0         29.91            10.0   
4       Interstate            37.0         62.0         30.40            10.0   
...            ...             ...          ...           ...             ...   
40335  large_local            33.8         41.0         30.27            10.0   
40336  large_local            69.8         60.0         30.20            10.0   
40337   Interstate            43.0        100.0         29.74             2.0   
40338      Beltway            37.0         79.0         29.81            10.0   
40339        local            89.0         48.0         29.27            10.0   

      Win

In [4]:
import numpy as np
import scipy
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

param_range = scipy.stats.loguniform(0.0001, 1000.0)

np.random.seed(1)
param_range.rvs(2)

pipe_svc = make_pipeline(OneHotEncoder(handle_unknown='ignore'),
                         StandardScaler(with_mean = False),
                         PCA(),
                         LinearSVC())

param_grid = [{'linearsvc__C': param_range}]

rs = RandomizedSearchCV(estimator=pipe_svc,
                        param_distributions=param_grid,
                        scoring='accuracy',
                        refit=True,
                        cv=2,
                        random_state=1,
                        n_jobs=1)


rs = rs.fit(X_train, y_train)
y_pred = rs.predict(X_test)
print(rs.best_score_)
print(rs.best_params_)

0.657436787307883
{'linearsvc__C': np.float64(0.00010018452045446543)}


In [5]:
from sklearn.metrics import classification_report
y_true = y_test
target_names = ['class 0 (Low Severity)', 'class 1 (High Severity)']
print(classification_report(y_true, y_pred, target_names=target_names))

                         precision    recall  f1-score   support

 class 0 (Low Severity)       0.66      0.75      0.70      5568
class 1 (High Severity)       0.70      0.60      0.65      5395

               accuracy                           0.68     10963
              macro avg       0.68      0.67      0.67     10963
           weighted avg       0.68      0.68      0.67     10963

